In [1]:
import datetime as dt
import pandas as pd

In [8]:
division = "Moreton"
neighbouring_division = {
    "Sydney": ["Sydney", "Grayndler"],
    "Wentworth": ["Sydney", "Wentworth", "Kingsford Smith"],
    "Bennelong": ["Bennelong", "North Sydney"],
    "Ryan": ["Ryan"],
    "Moreton": ["Moreton"]
}
df = pd.read_csv("/mnt/c/Users/marco/Downloads/prdelms.gaz.statics.250301.09.00.02.csv")
df = df[df["DivName"].str.contains(division) & (df["Status"] != "Abolition")]
df["PremisesName"] = df["PremisesName"].str.strip()

In [9]:
df_last_polling_place = pd.read_csv(
    "/mnt/c/Users/marco/Downloads/GeneralPollingPlacesDownload-27966.csv", skiprows=1
)

df_last_polling_place["PremisesNm"] = df_last_polling_place["PremisesNm"].replace(
    {"3A Joynton Avenue Creative Centre": "Joynton Avenue Creative Centre"}
)
df_last_polling_place_subset = df_last_polling_place[
    df_last_polling_place["PremisesNm"].isin(df["PremisesName"])
    & df_last_polling_place["DivisionNm"].isin(neighbouring_division[division])
]

df_last_votes = pd.read_csv(
    "/mnt/c/Users/marco/Downloads/HouseStateFirstPrefsByPollingPlaceDownload-27966-QLD.csv",
    skiprows=1,
)
df_last_votes_subset = df_last_votes[
    df_last_votes["PollingPlaceID"].isin(df_last_polling_place_subset["PollingPlaceID"])
].merge(
    df_last_polling_place_subset[["PremisesNm", "PollingPlaceID"]], on="PollingPlaceID"
)

parties = df_last_votes_subset["PartyNm"].unique()
def generate_party_mapping(parties):
    # Initialize the party categories
    party_mapping = {}

    # Categorize each party in the list based on the party name
    for party in parties:
        if "Labor" in party:
            party_mapping["Labor"] = party
        elif "Greens" in party:
            party_mapping["Greens"] = party
        elif "Liberal" in party:
            party_mapping["Liberal"] = party

    return party_mapping


# Generate the mapping dynamically
party_mapping = generate_party_mapping(parties)
Party_to_colour = {
    party_mapping["Labor"]: "red",
    party_mapping["Liberal"]: "blue",
    party_mapping["Greens"]: "green",
    "Other": "grey",
}


def pivot_table(df):

    votes = {
        party: df.loc[df["PartyNm"] == party, "OrdinaryVotes"].sum()
        for party in Party_to_colour
        if party != "Other"
    }

    votes.update(
        {
            "Other": df.loc[
                ~df["PartyNm"].isin(Party_to_colour.keys()), "OrdinaryVotes"
            ].sum()
        }
    )
    srs = pd.Series(votes)

    return srs


df_votes_by_party = (
    df_last_votes_subset.groupby("PremisesNm", group_keys=False)
    .apply(pivot_table)
    .reset_index()
    .rename(columns={"PremisesNm": "PremisesName"})
)
df_primary_votes = df.merge(df_votes_by_party, on="PremisesName", how="left")

/tmp/ipykernel_110182/2900436690.py:72: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(pivot_table)


In [10]:
# from geopy.geocoders import Nominatim
# from geopy.exc import GeocoderTimedOut

# # Sample list of addresses

# # Initialize geocoder
# geolocator = Nominatim(user_agent="address_mapper")


# # Function to get latitude and longitude
# def get_lat_lon(address):
#     try:
#         location = geolocator.geocode(address)
#         if location:
#             return location.latitude, location.longitude
#         else:
#             return None, None
#     except GeocoderTimedOut:
#         return None, None


# # Create a DataFrame

# # # Get coordinates
# # df["Latitude"], df["Longitude"] = zip(*df["PremisesName"].apply(get_lat_lon))
# # df_nan = df[df["Latitude"].isna()]
# # df_nan["Latitude"], df_nan["Longitude"] = zip(*(df_nan["Address1"] +df_nan["Address2"].astype("str").replace("nan", "") + df_nan["Address3"].astype("str").replace("nan", "") + " "+df_nan["Locality"]).apply(get_lat_lon))
# # assert(df_nan["Latitude"].isna().sum()==0)
# # valid_locations = pd.concat([df.dropna(subset=["Latitude", "Longitude"]), df_nan])

In [11]:
# Initialize a map centered around the first valid location
# valid_locations = df.dropna(subset=["Latitude", "Longitude"])

import folium
from branca.element import Template, MacroElement

if not df.empty:
    my_map = folium.Map(
        location=[df["Lat"].median(), df["Long"].median()], zoom_start=13.5
    )
else:
    print("No valid locations found.")
    exit()

median_voters = (df["OrdVoteEst"] + df["DecVoteEst"]).median()
# Add markers to the map
for _, row in df.iterrows():
    total_votes = row["OrdVoteEst"] + row["DecVoteEst"]
    radius = total_votes / 100  # Adjust scaling factor as needed

    wheelchair = row["WheelchairAccess"]
    if wheelchair == "Full":
        color = "blue"
    elif wheelchair == "Assisted":
        color = "grey"
    else:
        color = "red"

    popup_html = f"""
        <div style="font-family: Arial; font-size: 14px;">
            <b style="font-size: 16px; color: darkblue;">{row["PremisesName"]}</b><br>
            <b>Estimated Ordinary Votes:</b> {row["OrdVoteEst"]}<br>
            <b>Estimated Declaration Votes:</b> {row["DecVoteEst"]}<br>
            <b>Wheelchair Access:</b> {row["WheelchairAccess"]}
        </div>
    """

    folium.CircleMarker(
        location=[row["Lat"], row["Long"]],
        radius=radius,
        popup=folium.Popup(popup_html, max_width=300),
        fill=True,
        fill_color=color,
        color=color,
        fill_opacity=0.6,
    ).add_to(my_map)

# Legend HTML (for both color & size)

legend_html = """
<div style="position: fixed; 
            bottom: 40px; left: 40px; width: 250px; height: 240px; 
            background-color: white; z-index:9999; 
            font-size:14px; padding: 10px; 
            border-radius: 8px; box-shadow: 2px 2px 5px rgba(0,0,0,0.3);">
    <b>Polling Booth Legend</b><br>
    <b>Circle Color:</b><br>
    <div style="display: flex; align-items: center;">
        <div style="background:blue; width: 15px; height: 15px; border-radius: 50%; margin-right: 5px;"></div> 
        Full Wheelchair Access
    </div>
    <div style="display: flex; align-items: center;">
        <div style="background:grey; width: 15px; height: 15px; border-radius: 50%; margin-right: 5px;"></div> 
        Assisted Wheelchair Access
    </div>
    <div style="display: flex; align-items: center;">
        <div style="background:red; width: 15px; height: 15px; border-radius: 50%; margin-right: 5px;"></div> 
        No Wheelchair Access/No information
    </div>
    <b>Circle Size (Voter Estimate):</b><br>
    <div style="display: flex; align-items: center;">
        <div style="background:gray; width: 10px; height: 10px; border-radius: 50%; margin-right: 5px;"></div> 
        ~500 votes
    </div>
    <div style="display: flex; align-items: center;">
        <div style="background:gray; width: 20px; height: 20px; border-radius: 50%; margin-right: 5px;"></div> 
        ~1,000 votes
    </div>
    <div style="display: flex; align-items: center;">
        <div style="background:gray; width: 40px; height: 40px; border-radius: 50%; margin-right: 5px;"></div> 
        ~2,000 votes
    </div>
</div>
"""

# Add legend to the map
legend = MacroElement()
legend._template = Template(f"""
    {{% macro html(this, kwargs) %}}
    {legend_html}
    {{% endmacro %}}
""")

my_map.get_root().add_child(legend)


# Save map to an HTML file
# my_map.save(f"Division_of_Sydney_expected_polling_day_locations_{dt.datetime.now()}.html")
my_map.save(f"Division_of_{division}_expected_polling_day_locations.html")
# my_map.save(f"Division_of_Ryan_expected_polling_day_locations.html")
print("Map has been saved as map.html. Open it in your browser.")

Map has been saved as map.html. Open it in your browser.


In [12]:
import folium
from branca.element import Template, MacroElement
import pandas as pd
import matplotlib.pyplot as plt
import io
import base64
import numpy as np


# Initialize map
if not df_primary_votes.empty:
    my_map = folium.Map(
        location=[df["Lat"].median(), df["Long"].median()], zoom_start=13.5
    )
else:
    print("No valid locations found.")
    exit()


# Function to create pie chart as an SVG
def create_pie_chart(row, radius):
    """Generates a pie chart image as an SVG string."""
    diameter = radius * 2
    scale_factor = 2
    px = 1 / plt.rcParams["figure.dpi"]
    fig, ax = plt.subplots(
        figsize=(diameter * px * scale_factor, diameter * px * scale_factor)
    )  # Adjust size for better visibility
    values = [row.get(party, 0) for party in Party_to_colour.keys()]
    colors = list(Party_to_colour.values())

    ax.pie(
        values,
        labels=None,
        colors=colors,
        startangle=90,
        # wedgeprops={"edgecolor": "black"},
    )
    ax.set_xticks([])
    ax.set_yticks([])

    # Convert plot to SVG
    buff = io.StringIO()
    plt.savefig(buff, format="svg", bbox_inches="tight", transparent=True)
    buff.seek(0)

    svg = buff.getvalue()
    plt.close(fig)

    return svg


# Add markers
for _, row in df_primary_votes.iterrows():
    total_votes = row["OrdVoteEst"] + row["DecVoteEst"]
    radius = total_votes / 100  # Adjusted scaling for better visualization

    votes = [
        f"<b>Last election {party} primary:</b> {row[party]}<br>"
        for party in Party_to_colour.keys()
    ]
    popup_html = f"""
        <div style="font-family: Arial; font-size: 14px;">
            <b style="font-size: 16px; color: darkblue;">{row["PremisesName"]}</b><br>
            <b>Estimated Ordinary Votes:</b> {row["OrdVoteEst"]}<br>
            <b>Estimated Declaration Votes:</b> {row["DecVoteEst"]}<br>
            <b>Wheelchair Access:</b> {row["WheelchairAccess"]}<br>
            {"".join(votes)}
        </div>
    """

    if np.isnan(row[party_mapping["Labor"]]):  # No party votes recorded
        color = "grey"
        folium.CircleMarker(
            location=[row["Lat"], row["Long"]],
            radius=radius,
            popup=folium.Popup(popup_html, max_width=300),
            fill=True,
            fill_color=color,
            color=color,
            fill_opacity=0.6,
        ).add_to(my_map)
    else:
        # Generate pie chart SVG
        pie_chart_svg = create_pie_chart(row, radius)

        icon_html = f"""
        <div>
            {pie_chart_svg}
        </div>
        """

        folium.Marker(
            location=[row["Lat"], row["Long"]],
            popup=folium.Popup(popup_html, max_width=300),
            icon=folium.DivIcon(
                html=icon_html, icon_size=(radius * 2, radius * 2)
            ),  # Embed SVG
        ).add_to(my_map)

# Legend HTML
legend_html = """
<div style="position: fixed; 
            bottom: 40px; left: 40px; width: 280px; height: auto; 
            background-color: white; z-index:9999; 
            font-size:14px; padding: 10px; 
            border-radius: 8px; box-shadow: 2px 2px 5px rgba(0,0,0,0.3);">
    <b>Polling Booth Legend</b><br>
    <b>Circle Size (Voter Estimate):</b><br>
    <div style="display: flex; align-items: center;">
        <div style="background:gray; width: 10px; height: 10px; border-radius: 50%; margin-right: 5px;"></div> 
        ~500 votes
    </div>
    <div style="display: flex; align-items: center;">
        <div style="background:gray; width: 20px; height: 20px; border-radius: 50%; margin-right: 5px;"></div> 
        ~1,000 votes
    </div>
    <div style="display: flex; align-items: center;">
        <div style="background:gray; width: 40px; height: 40px; border-radius: 50%; margin-right: 5px;"></div> 
        ~2,000 votes
    </div>
    <b>Party Colors:</b><br>
    <div style="display: flex; align-items: center;">
        <div style="background:red; width: 15px; height: 15px; border-radius: 50%; margin-right: 5px;"></div> Labor
    </div>
    <div style="display: flex; align-items: center;">
        <div style="background:blue; width: 15px; height: 15px; border-radius: 50%; margin-right: 5px;"></div> Liberal
    </div>
    <div style="display: flex; align-items: center;">
        <div style="background:green; width: 15px; height: 15px; border-radius: 50%; margin-right: 5px;"></div> The Greens
    </div>
    <div style="display: flex; align-items: center;">
        <div style="background:grey; width: 15px; height: 15px; border-radius: 50%; margin-right: 5px;"></div> Other
    </div>
</div>
"""

# Add legend to the map
legend = MacroElement()
legend._template = Template(f"""
    {{% macro html(this, kwargs) %}}
    {legend_html}
    {{% endmacro %}}
""")

my_map.get_root().add_child(legend)

# Save map
my_map.save(
    f"Division_of_{division}_expected_polling_day_locations_primary_vote_last_election.html"
)
print("Map has been saved as 'map.html'. Open it in your browser.")


Map has been saved as 'map.html'. Open it in your browser.
